In [19]:
from langfuse import Langfuse

langfuse = Langfuse(
    secret_key="sk-lf-a65b23f9-0ffb-4063-8c09-5e1c265e9c4a",
    public_key="pk-lf-c5d11bfe-392e-4ae1-ad90-aa4a8639965c",
    host="https://cloud.langfuse.com"
  # host="https://us.cloud.langfuse.com", # 🇺🇸 US region
)


# langfuse = Langfuse()


In [30]:
import json
from langchain_core.documents import Document
from tqdm.auto import tqdm

def create_convfinqa_langfuse_dataset(filepath, name, description, limit: int = None):
    langfuse.create_dataset(name=name, description=description)
    with open(filepath, 'r') as f:
        data = json.load(f)
    docs = []
    
    QA_FIELDS = ["qa", *[f"qa_{i}" for i in range(10)]]
    SKIPPED_METADATA_FIELDS = ['annotation', *QA_FIELDS]
    if limit:
        data = data[:limit]

    for entry in tqdm(data):
        # Combine pre_text, post_text, and table content into a single text block

        # Loop through every available QA field in the entry
        for qa_field in set(QA_FIELDS).intersection(entry.keys()):
            # Upload to Langfuse
            langfuse.create_dataset_item(
                dataset_name=name,
                # any python object or value
                input=entry[qa_field]["question"],
                # any python object or value, optional
                expected_output=entry[qa_field]["answer"],
                metadata={
                    "document": {field: entry[field] for field in entry.keys() if field not in SKIPPED_METADATA_FIELDS},
                }
            )
  
        # # Process the table to include in the text block
        table_text = []
        for row in entry['table']:
            # Join each cell in the row with a tab for clarity
            table_text.append("\t".join(row))
    
    return docs


In [31]:
DATA_PATH = 'ConvFinQA/data/train.json'
docs = create_convfinqa_langfuse_dataset(DATA_PATH, "convfinqa-train", "Dataset created from ConvFinQA train data", limit=1000)
print(docs)
# vector_store.add_documents(docs)

['qa', 'qa_0', 'qa_1', 'qa_2', 'qa_3', 'qa_4', 'qa_5', 'qa_6', 'qa_7', 'qa_8', 'qa_9']
[]
